## Goal

This notebook describes some of the tools I find useful when developing analytics projects.

### Switchboard

Decentralized oracle provider. Anybody can publish/read from Oracle.

https://github.com/switchboard-xyz/switchboard-v2/tree/main/libraries/py

In [8]:
import asyncio
from solana.keypair import Keypair
from solana.publickey import PublicKey
from solana.rpc.async_api import AsyncClient
from anchorpy import Program, Provider, Wallet
from switchboardpy import AggregatorAccount, AccountParams
from switchboardpy import (
  SBV2_DEVNET_PID,
  AccountParams,
  OracleAccount,
  OracleInitParams,
  OracleWithdrawParams,
)

In [10]:
# Devnet Program ID.
SBV2_DEVNET_PID = PublicKey(
    '2TfB33aLaneQb5TNVwyDz3jSZXS6jdW2ARw1Dgf84XCG'
)
client = AsyncClient("https://api.devnet.solana.com/")
provider = Provider(client, Wallet(Keypair()))
program = await Program.at(
    SBV2_DEVNET_PID, provider
)
#agg = AggregatorAccount(AccountParams(
#    program=program, public_key=PublicKey("88FX4tBstuwBPNhQU4EEBoPX35neSu4Le9zDSwtPRRQz")))

agg = AggregatorAccount(AccountParams(program=program, public_key=PublicKey("GvDMxPzN1sCj7L26YDK2HnMRXEQmQ2aemov8YBtPS7vR")))

# getting aggregator data
data = await agg.load_data()

# getting most recent value (decimal.Decimal)
val = await agg.get_latest_value()

print('LATEST VALUE (SOL/USDC):')
print(val)

#await program.close()

LATEST VALUE (SOL/USDC):
39.9073563125


Further data feeds can be found here
https://switchboard.xyz/explorer/2/GvDMxPzN1sCj7L26YDK2HnMRXEQmQ2aemov8YBtPS7vR

### Pyth

Solana-focused oracle. Makes some smart curation of oracles and provides a mean price as well as a confidence interval.



In [11]:
from pythclient.pythclient import PythClient
from pythclient.pythaccounts import PythPriceAccount
from pythclient.utils import get_key
from pythclient.pythaccounts import PythPriceAccount, PythPriceStatus
from pythclient.solana import SolanaClient, SolanaPublicKey, SOLANA_DEVNET_HTTP_ENDPOINT, SOLANA_DEVNET_WS_ENDPOINT

In [12]:
solana_network="devnet"
use_program = None
c = PythClient(first_mapping_account_key=get_key(solana_network, "mapping"),
program_key=get_key(solana_network, "program") if use_program else None)
await c.refresh_all_prices()
products = await c.get_products()
for p in products:
    print(p.attrs)
    prices = await p.get_prices()
    for _, pr in prices.items():
        
        print(
            pr.price_type,
            pr.aggregate_price_status,
            pr.aggregate_price,
            "p/m",
            pr.aggregate_price_confidence_interval,
        )
        
symbols = ['Crypto.SOL/USD', 'Crypto.ETH/USD','Crypto.BTC/USD']

{'symbol': 'Crypto.BCH/USD', 'asset_type': 'Crypto', 'quote_currency': 'USD', 'description': 'BCH/USD', 'generic_symbol': 'BCHUSD', 'base': 'BCH'}
PythPriceType.PRICE PythPriceStatus.TRADING 133.99736019 p/m 0.01263981
{'symbol': 'Crypto.LTC/USD', 'asset_type': 'Crypto', 'quote_currency': 'USD', 'description': 'LTC/USD', 'generic_symbol': 'LTCUSD', 'base': 'LTC'}
PythPriceType.PRICE PythPriceStatus.TRADING 58.776203280000004 p/m 0.00909672
{'symbol': 'Equity.US.AAPL/USD', 'asset_type': 'Equity', 'quote_currency': 'USD', 'description': 'APPLE INC', 'base': 'AAPL', 'country': 'US', 'cms_symbol': 'AAPL', 'cqs_symbol': 'AAPL', 'nasdaq_symbol': 'AAPL'}
PythPriceType.PRICE PythPriceStatus.UNKNOWN None p/m None
{'symbol': 'Equity.US.TSLA/USD', 'asset_type': 'Equity', 'quote_currency': 'USD', 'description': 'TESLA INC', 'base': 'TSLA', 'country': 'US', 'cms_symbol': 'TSLA', 'cqs_symbol': 'TSLA', 'nasdaq_symbol': 'TSLA'}
PythPriceType.PRICE PythPriceStatus.UNKNOWN None p/m None
{'symbol': 'Equi

In [17]:
eth_account = 'EdVCmQ9FSPcVe5YySXDPCRmc8aDQLKJ9xvYBMZPie1Vw'
account_key = SolanaPublicKey(eth_account)
solana_client = SolanaClient(endpoint=SOLANA_DEVNET_HTTP_ENDPOINT, ws_endpoint=SOLANA_DEVNET_WS_ENDPOINT)
price: PythPriceAccount = PythPriceAccount(account_key, solana_client)

await price.update()
price_status = price.aggregate_price_status
if price_status == PythPriceStatus.TRADING:
    # Sample output: "DOGE/USD is 0.141455 ± 7.4e-05"
    print("Price ETH/USD is", price.aggregate_price, "±", price.aggregate_price_confidence_interval)
else:
    print("Price is not valid now. Status is", price_status)
    
await solana_client.close()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f20b55ed820>


Price ETH/USD is 1701.5414766000001 ± 1.0514766
